In [1]:
import pymongo
from pymongo import MongoClient
import redis
from random import randint
import datetime
import pandas as pd

In [2]:
client = MongoClient()
r = redis.StrictRedis(host='bdl1.eng.tau.ac.il', port=6379)
db = client['stud22']
collection = db.recruiting

In [3]:
def clearRedis():
    for key in r.keys('stud22:*'):
        r.delete(key)

In [4]:
#clean up the collection before starting the exercise
def clearDBs():
    db.recruiting.drop()
    clearRedis()

In [5]:
def add_company(companyRecord):
    result=collection.find_one({"company_name": companyRecord['company_name']})
    if result is None:
        companyRecord['jobs']=[]
        collection.insert_one(companyRecord)
        print('Company '+companyRecord['company_name']+' added successfully')
    else:
        print('Company '+companyRecord['company_name']+' already exists')

In [6]:
def add_job(jobRecord,compName):
    result=collection.find_one({"company_name": compName})
    if result is None:
        print('Company '+compName+' not  found')
    else:
        jobRecord['candidates']=[]
        maxJobId=r.incr('stud22:jobId')
        jobRecord['jobId']=maxJobId#+1
        collection.update_one({"company_name": compName}, {'$push': {'jobs': jobRecord}})
        r.incr('stud22:jobCount:'+compName)

In [7]:
def application(candidate, application_time, job_id):
    comp=collection.find_one({"jobs.jobId": int(job_id)})
    if comp is None:
        print 'Job id '+job_id+' not found'
        return
    #check if the candidate's email arleady exists in the candidatesInJob set
    if r.sismember('stud22:candidatesInJob:'+job_id, candidate["email"]):
        print 'Candidate with email '+candidate['email']+' already applied to job id '+job_id
        return
    
    #parse the application_time from string to date
    candidate['applicationTime']=datetime.datetime.strptime(application_time,"%d-%m-%Y %H:%M:%S")
    
    #find the company document and the relevant job in its jobs array, add to it's nested Candidate
    #array the candidate's record
    collection.update_one({"company_name": comp['company_name'], "jobs.jobId": int(job_id)}, {"$push": {"jobs.$.candidates": candidate}})
    
    #add the candidate's email to the candidatesInJob set
    r.sadd('stud22:candidatesInJob:'+job_id,candidate['email'])
    #add the company to the candidate list of companies applied to, but first remove it if already exists
    r.lrem('stud22:candidate:'+candidate['email'],-1,comp['company_name'])
    r.lpush('stud22:candidate:'+candidate['email'],comp['company_name'])

In [8]:
def show_latest_applications(k, candidate_email):
    #if the number of last companies is smaller than the number of companies the candidate applied to, make
    #k equal the number of jobs applied to (to prevent the lrange from not working correctly)
    if k>r.llen('stud22:candidate:'+candidate_email):
        k=r.llen('stud22:candidate:'+candidate_email)
    if k==0:
        print candidate_email+' wasn\'t found'
        return
    for comp in r.lrange('stud22:candidate:'+candidate_email,0,k-1):
        print comp

In [9]:
def show_candidates(job_id, skills):
    #find the company with the relevant jobId and that at least one candidate has the required skills
    comp=collection.find({"jobs.jobId":int(job_id),"jobs.candidates.skills": {"$all": skills} })
    for c in comp:
        for job in c['jobs']:
            if job['jobId']==int(job_id):
                for candidate in job['candidates']:
                    foundSkills=0
                    for skill in skills:
                        if skill in candidate['skills']:
                            foundSkills=foundSkills+1
                    if foundSkills==len(skills):
                        print candidate['email']

In [10]:
def count_jobs_by_company():
    results=[]
    #results.append({'Company','Jobs'})
    for key in r.keys('stud22:jobCount:*'):
        compName=key.replace('stud22:jobCount:','')
        #print compName+'\t\t'+str(r.get(key))
        results.append({'Company Name': compName,'Jobs': r.get(key)})
    df = pd.DataFrame(results)
    display(df)

In [11]:
def count_candidates_by_job():
    aggResult=collection.aggregate([
        {"$unwind": "$jobs"},
        {"$unwind": "$jobs.candidates"},
        {"$match": {"jobs.candidates.applicationTime": {"$gte": datetime.datetime.now() + datetime.timedelta(-30)} }},
        {"$group": {
                "_id": {
                    "JobId": "$jobs.jobId",
                },
                "total": { 
                    "$sum": 1
                }
            }
        },
        {"$sort":{"_id.JobId":1}}
    ])

    results=[]
    for agg in aggResult:
        results.append({'Job Id': agg['_id']['JobId'],'Candidates': agg['total']})
    df = pd.DataFrame(results)
    df=df[['Job Id', 'Candidates']]
    display(df)

The function below prints the entire collection by iterating over the entire mongoDB collection and printing the companies, jobs and candidates (along with their skills) in a structured, easy to ready format

In [12]:
def printCollection():
    result=collection.find()
    for comp in result:
        print 'Company '+comp['company_name']
        for job in comp['jobs']:
            jobStr='\tjob '+str(job['jobId'])+' - '+job['job_name']+'('
            for skill in job['skills']:
                jobStr=jobStr+skill+','
            print jobStr[:-1]+'):'
            for candidate in job['candidates']:
                pairsSet=set()
                candStr='\t\tcandidate '+candidate['email']+', skills: ('
                skills=candidate['skills']
                skills.sort()
                for skill in skills:
                    candStr=candStr+skill+','
                    #calculate all the pairs of skills so we can later use it in the execute function
                    #(this is not needed for the actual program but rather for the testing so we can
                    #search for the skills pair with the most candidates)
                    for skillAgain in candidate['skills']:
                        skillStr=skill+'__'+skillAgain
                        skillStrSwitched=skillAgain+'__'+skill
                        if skill!=skillAgain and not(skillStrSwitched in pairsSet):
                             pairsSet.add(skillStr)
                for pair in pairsSet:
                    r.incr('stud22:candidatesPerJobBySkills:'+str(job['jobId'])+':'+pair)
                print candStr[:-1]+')'

The function below populates the data model with random values by generating random company names, random jobs with random required skills and random candidates with random skills. Both the names of the companies, jobs and candidates are random as is the number of jobs per company and number of candidates per job and the number of skills in each job/candidate

In [13]:
def populateDB():
    clearDBs()
    compPrefix=['Blue','Red','Super','Star','Gold','Wood','Shield','Smart','Clear']
    compSuffix=['Tech','Investing','Sound','Innovations','Cookies','Net','Alliance']
    jobsPrefix=['BI','Finance','Information Systems','R&D','Sales','Marketing','Presales','Customer Succes','Support']
    jobsSuffix=['Development','Analyst','Senior Analyst','Team Leader','Expert','Lead','Operations','Manager']
    firstNames=['Tal','Anna','Rami','Lior','Meital','Adi','Oren','Raz','Amit','Liat','David','Mor','Sivan','Ran','Dan']
    lastNames=['Cohen','Levi','Carmi','Ben David','Gez','Butbul','Katz','Ashkenazi','Mizrahi','Smith','Bergman','Goldman']
    skills=['Python','Big Data','Hadoop','R','Java','Front End','node.js','SQL','Analysis','Excel','mongodb','redis','Kafka','ETL','CPA','Jenkins']
    
    candidatesDict={}
    for c in range(0,20):
        candName=firstNames[randint(0, len(firstNames)-1)]+' '+lastNames[randint(0, len(lastNames)-1)]
        candEmail=candName.replace(' ','.')+'@gmail.com'
        candLinkedIn='https://www.linkedin.com/in/'+candName.replace(' ','.')+'/'
        skillsCount=randint(1,4)
        candSkills=set()
        for k in range(0,skillsCount):
            candSkills.add(skills[randint(0,len(skills)-1)])
        candidatesDict[candEmail]={'candidate_name':candName,'email':candEmail,'linkedin':candLinkedIn, 'skills': list(candSkills)}
            
    candidates=list(candidatesDict.values())
    for i in range(0, 12):
        compName=compPrefix[randint(0, len(compPrefix)-1)]+' '+compSuffix[randint(0, len(compSuffix)-1)]
        add_company({'company_name':compName, 'company_description':'Stam'})
        jobsNum=randint(0,8)
        for j in range(0,jobsNum):
            jobName=jobsPrefix[randint(0, len(jobsPrefix)-1)]+' '+jobsSuffix[randint(0, len(jobsSuffix)-1)]
            skillsCount=randint(1,4)
            jobSkills=set()
            for k in range(0,skillsCount):
                jobSkills.add(skills[randint(0,len(skills)-1)])
            add_job({'job_name':jobName, 'location': 'Tel Aviv','skills':list(jobSkills),'status':'open','publish_date':'01-02-2019'},compName)
            candidatesNum=randint(0,10)
            if candidatesNum>8:
                candidatesNum=0
            for k in range(0,candidatesNum):
                appDate=datetime.datetime.now() + datetime.timedelta(randint(-60,0))
                appDateStr=appDate.strftime("%d-%m-%Y %H:%M:%S")
                application(candidates[randint(0,len(candidates)-1)],appDateStr, r.get('stud22:jobId'))

In [14]:
def redisToStr():
    redisStr=''
    keys=r.keys('stud22:*')
    keys.sort()
    for key in keys:
        try:
            redisStr=redisStr+key+str(r.get(key))
        except:
            redisStr=redisStr+key
            try:
                members=list(r.smembers(key))
                members.sort()
                for member in members:
                    redisStr=redisStr+str(member)
            except:
                members=list(r.lrange(key,0,-1))
                members.sort()
                for member in members:
                    redisStr=redisStr+str(member)
    return redisStr

In [15]:
def validateRedisRecovery():
    #Clear the keys added just for the sake of the test, they are not part of the recovery
    #and thus shouldn't be part of the recovery validation
    for key in r.keys('stud22:candidatesPerJobBySkills:*'):
        r.delete(key)
    #Stringify Redis before recovery
    originalRedisString=redisToStr()
    #Delete all Redis keys
    clearRedis()
    #Run the recovery process
    recovery()
    #Stringy the recoverd Redis
    newRedisString=redisToStr()
    #Compare strings to validate the recovery
    if originalRedisString==newRedisString:
        print 'Recovery verification succeeded!'
    else:
        print 'Reovery verification failed!'

In [16]:
def recovery():
    result=collection.find()
    for comp in result:
        for job in comp['jobs']:
            maxJobId=r.incr('stud22:jobId')
            r.incr('stud22:jobCount:'+comp['company_name'])
            for candidate in job['candidates']:
                #add the candidate's email to the candidatesInJob set
                r.sadd('stud22:candidatesInJob:'+str(job['jobId']),candidate['email'])
                #add the company to the candidate list of companies applied to, but first remove it if already exists
                r.lrem('stud22:candidate:'+candidate['email'],-1,comp['company_name'])
                r.lpush('stud22:candidate:'+candidate['email'],comp['company_name'])

In [17]:
def execute():
    clearDBs()
    populateDB()
    printCollection()
    print ('Random data population completed')
    print ('***********************************************************\n\n')
    maxCount=0
    maxCandidateEmail=''
    for key in r.keys('stud22:candidate:*'):
        curCount=len(r.lrange(key,0,-1))
        if curCount>maxCount:
            maxCount=curCount
            maxCandidateEmail=key.replace('stud22:candidate:','')
    if maxCount>1:
        maxCount=maxCount-1
    print 'Last '+str(maxCount)+' companies '+maxCandidateEmail+' applied to:'
    show_latest_applications(maxCount, maxCandidateEmail)
    print('\n\n')
    maxCount=0
    maxJobId=0
    maxSkillPair=''
    for key in r.keys('stud22:candidatesPerJobBySkills:*'):
        cleanKey=key.replace('stud22:candidatesPerJobBySkills:','')
        if (r.get(key)>maxCount):
            maxCount=r.get(key)
            maxJobId=cleanKey.split(':')[0]
            maxSkillPair=cleanKey.split(':')[1]
    pairStr='('+maxSkillPair.split('__')[0]+', '+maxSkillPair.split('__')[1]+')'
    print 'Candidates for job #'+str(maxJobId)+' with skills '+pairStr+':'
    show_candidates(str(maxJobId),maxSkillPair.split('__'))
    print '\n\n# of Jobs by Company:'
    count_jobs_by_company()
    print '\n\n# of Candidates by Job:'
    count_candidates_by_job()
    print '\n\nData manipulation and reporting completed, moving to recovery'
    validateRedisRecovery()

In [18]:
execute()

Company Clear Alliance added successfully
Candidate with email David.Levi@gmail.com already applied to job id 8
Company Star Net added successfully
Company Wood Sound added successfully
Candidate with email Meital.Goldman@gmail.com already applied to job id 10
Candidate with email Meital.Goldman@gmail.com already applied to job id 10
Candidate with email Adi.Ben.David@gmail.com already applied to job id 12
Candidate with email Sivan.Ben.David@gmail.com already applied to job id 12
Candidate with email Sivan.Smith@gmail.com already applied to job id 13
Candidate with email Mor.Gez@gmail.com already applied to job id 13
Candidate with email Amit.Butbul@gmail.com already applied to job id 15
Candidate with email Sivan.Smith@gmail.com already applied to job id 16
Company Wood Cookies added successfully
Company Smart Alliance added successfully
Candidate with email Adi.Ben.David@gmail.com already applied to job id 18
Candidate with email Adi.Katz@gmail.com already applied to job id 22
Candi

,Company Name,Jobs
0,Super Innovations,4
1,Smart Tech,2
2,Gold Cookies,5
3,Clear Innovations,15
4,Star Tech,8
5,Wood Cookies,1
6,Star Net,1
7,Wood Sound,14
8,Smart Alliance,8
9,Clear Alliance,8




# of Candidates by Job:


,Job Id,Candidates
0,1,2
1,2,1
2,3,1
3,4,1
4,6,3
5,7,7
6,8,3
7,9,2
8,10,2
9,13,1




Data manipulation and reporting completed, moving to recovery
Recovery verification succeeded!
